###**gamma_mod_ml**

This notebook extends findings from Jones et al., 2020, which showed that 40-Hz tACS, but not tDCS, increases gamma power during auditory click trains at the group level. There are two overarching goals:

1. **Characterize gamma-band steady-state evoked potentials (SSEPs) and extract features for machine learning.** Using spectral decomposition (specparam), bandpass filtering, time-domain analysis (response latency, amplitude dynamics), and cycle-by-cycle waveform analysis (bycycle), extract comprehensive features spanning frequency-domain, temporal, and waveform morphology characteristics. Engineer baseline-relative features to capture stimulation-induced change while controlling for inter-individual baseline differences, creating a 14-feature dataset suitable for single-trial classification of stimulation conditions (tACS, tDCS, Sham).

2. **Classify stimulation conditions using data from a single channel (Cz).** Compare classification algorithms (random forest, gradient boosting, logistic regression, SVM) using a 70/30 train/test split rather than leave-one-subject-out cross-validation to account for individual differences in stimulation response. Optimize hyperparameters using 2-stage randomized search, identify optimal features through importance ranking, and validate model performance with permutation testing. Demonstrate that within-subject baseline-relative features enable single-trial classification with ~2x better-than-chance accuracy (0.668 vs. 0.333), with cycle-by-cycle waveform features (amplitude) dominating spectral power measures.

**Key scientific findings**:
- **Inter-individual variability in stimulation effects**: Leave-one-subject-out cross-validation failed (performance ~0.4), confirming that stimulation responses are  individual-specific and may not generalize across subjects without personalized calibration.

- **Within-subject normalization of single trials**: Baseline-relative features control for between-subject differences in baseline variability while preserving single-trial resolution.

- **Importance of waveform dynamics**: Cycle-by-cycle amplitude (23.6% feature importance) dominates traditional spectral measures like peak power (4.1% importance), suggesting that oscillatory waveform shape captures stimulation effects more sensitively than conventional spectral analysis at the single-trial level.

- **Non-linear decision boundaries**: Tree-based models substantially outperform linear classifiers, indicating complex, non-linear relationships between EEG features and stimulation conditions.

- **Real-world deployment feasibility**: The within-subject calibration strategy mirrors realistic scenarios where baseline data are collected before classifying ongoing stimulation.

These results support the feasibility of real-time, personalized brain stimulation approaches where EEG signatures are used to adaptively monitor and optimize stimulation parameters on an individual basis.

Copyright (c) 2025  
EL Johnson, PhD

###Import modules:

In [ ]:
import gdown
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

!pip install mne  # for custom modules
!pip install specparam
!pip install bycycle

# custom modules
from gamma_utils import load_eeg_files, compute_psd, fit_specparam, extract_ssep, \
  analyze_ssep, analyze_bycycle, create_paired_df
from plotting_utils import plot_psd, plot_specparam, plot_ssep, plot_ssep_temporal, plot_bycycle
from ml_utils import compare_models, tune_gradient_boosting, rank_features, \
  test_feature_subsets, permutation_test

###Download data from Google Drive:

In [ ]:
# download
fid_tacs = '1jcZDPJbV7fPh85R3G9raQf2MpvHbfeWQ'
fid_tdcs = '1smAHUQBAF3WVL32t7jThWef03SLqKQQ1'
fid_sham = '13xCr3RUNQeoVr6SLdQw3N_gkTbXvm_rR'
fid_baseline = '1r_DP-FjMUWuRZ4dgWnIYKggcbupcLtEk'

url_tacs = f'https://drive.google.com/uc?id={fid_tacs}'
url_tdcs = f'https://drive.google.com/uc?id={fid_tdcs}'
url_sham = f'https://drive.google.com/uc?id={fid_sham}'
url_baseline = f'https://drive.google.com/uc?id={fid_baseline}'

gdown.download(url_tacs, 'tacs.zip', quiet = False)
gdown.download(url_tdcs, 'tdcs.zip', quiet = False)
gdown.download(url_sham, 'sham.zip', quiet = False)
gdown.download(url_baseline, 'baseline.zip', quiet = False)

# unzip
with zipfile.ZipFile('tacs.zip', 'r') as zip_ref:
  zip_ref.extractall()

with zipfile.ZipFile('tdcs.zip', 'r') as zip_ref:
  zip_ref.extractall()

with zipfile.ZipFile('sham.zip', 'r') as zip_ref:
  zip_ref.extractall()

with zipfile.ZipFile('baseline.zip', 'r') as zip_ref:
  zip_ref.extractall()

### Load the data

Each MATLAB input file contains one data structure per subject (`data_clean`), which is the output of FieldTrip's `ft_preprocessing`. Each structure contains the following variables of interest:
- `label` = channel names
- `time` = time vectors for each trial (sec)
- `trial` = EEG data (channels x timepoints per epoch)
- `fsample` = sampling rate (Hz)

The `load_eeg_files()` function converts these FieldTrip structures directly into MNE `Epochs` objects. Each output is a dictionary containing an MNE `Epochs` object (e.g., `KS`), preserving all channel info, trial data, and sampling parameters from the original MATLAB structure.

In [ ]:
all_tacs = load_eeg_files(data_dir = '/content/tacs')
all_tdcs = load_eeg_files(data_dir = '/content/tdcs')
all_sham = load_eeg_files(data_dir = '/content/sham')
all_baseline = load_eeg_files(data_dir = '/content/baseline')

In [ ]:
all_tacs['KS']  # take a look

### Goal 1.1: Compute and visualize power spectral density (PSD)

**Analysis overview**: The `compute_psd()` function calculates power spectral density (1-55 Hz) using the multitaper method applied to the 500-ms stimulus window. PSD is computed for each trial at channel Cz, then averaged across trials within each subject. The `plot_psd()` function visualizes grand-averaged PSD (mean +/- SEM across subjects) for each treatment condition compared to its paired baseline.

**Key parameters**:
- Frequency range: 1-55 Hz captures both low-frequency and gamma-band activity
- Bandwidth: 2 Hz provides narrowband spectral smoothing
- Zero-padding: Extended to 2048 samples for 0.25 Hz frequency resolution
- Normalization: PSD in power/Hz, converted to dB for visualization

**Results**: The plots show clear 40 Hz peaks in all conditions, reflecting the 40 Hz steady-state evoked potential (SSEP) to the click train stimulus. Comparing post-treatment to baseline:
- tACS vs. baseline: The 40 Hz peak is enhanced in tACS relative to baseline, consistent with the published finding that 40 Hz tACS enhances gamma activity.
- tDCS vs. baseline: Near-identical profiles, consistent with the published finding that tDCS does not enhance gamma activity.
- Sham vs. baseline: Near-identical profiles, confirming stability of the SSEP in the control condition.

In [ ]:
psd_tacs = compute_psd(all_tacs)
psd_tdcs = compute_psd(all_tdcs)
psd_sham = compute_psd(all_sham)
psd_baseline = compute_psd(all_baseline)

In [ ]:
plot_psd(psd_baseline, psd_sham, psd_tacs, psd_tdcs)

### Goal 1.2: Parameterize power spectra and extract frequency-domain features

**Analysis overview**: The `fit_specparam()` function decomposes each trial's PSD into periodic (oscillatory) and aperiodic (1/f-like background) components using the specparam algorithm. The model fits a peak in the 30-50 Hz range, identifying the peak closest to 40 Hz. For each detected peak, the function extracts center frequency, power, bandwidth, and signal-to-noise ratio (SNR). Aperiodic parameters (offset and slope) characterize the 1/f-like background across the full 1-55 Hz range. The `plot_specparam()` function visualizes distributions of all six specparam features for each treatment condition compared to its paired baseline.

**Key parameters**:
- Frequency range: 1-55 Hz for aperiodic fitting, 30-50 Hz for peak detection
- Peak width limits: 1-10 Hz allows for natural variation in SSEP bandwidth
- Minimum peak height: 0.1 threshold for reliable peak detection
- Peak selection: Closest to 40 Hz within the 30-50 Hz gamma band

**Results**: The histograms show distributions of specparam features across all trials and subjects. Comparing post-treatment to baseline:
- **Peak frequency**: All conditions show tight distributions centered near 40 Hz, reflecting the stimulus-locked SSEP.
- **Peak power**: tACS shows increased peak power relative to baseline, consistent with enhanced gamma oscillations.
- **Peak bandwidth**: All conditions show majority narrow bandwidths, characteristic of stimulus-driven oscillations.
- **Peak SNR**: tACS shows decreased SNR relative to baseline, consistent with lower oscillatory activity relative to the aperiodic background.
- **Aperiodic offset**: Minimal differences across conditions, suggesting stable overall power.
- **Aperiodic slope**: All conditions show negative slopes characteristic of neural 1/f-like activity.

The observation of increased peak power and decreased SNR with tACS suggests that tACS enhances both oscillatory and aperiodic activity ~40 Hz, with greater enhancement of aperiodic activity.

In [ ]:
specparam_tacs = fit_specparam(psd_tacs)
specparam_tdcs = fit_specparam(psd_tdcs)
specparam_sham = fit_specparam(psd_sham)
specparam_baseline = fit_specparam(psd_baseline)

In [ ]:
plot_specparam(specparam_baseline, specparam_sham, specparam_tacs, specparam_tdcs)

### Goal 1.3: Compute and visualize bandpass-filtered SSEP signals

**Analysis overview**: The `extract_ssep()` function isolates trial-specific gamma oscillations by applying a bandpass filter centered on each trial's detected peak frequency and bandwidth from specparam. For each trial, a second-order Butterworth filter is applied to the raw signal at channel Cz. The `plot_ssep()` function visualizes example filtered SSEP traces from one trial per condition compared to baseline.

**Key parameters**:
- Filter range: Defined by peak center frequency and bandwidth from specparam
- Filter design: Second-order Butterworth bandpass filter for stable frequency response
- Zero-phase filtering: Forward-backward filtering preserves temporal features

**Results**: The example traces illustrate the filtered ~40 Hz oscillations over the 500-ms stimulus window. The plots show sustained oscillatory activity across all conditions, with trial-to-trial variability.

In [ ]:
ssep_tacs = extract_ssep(all_tacs, specparam_tacs)
ssep_tdcs = extract_ssep(all_tdcs, specparam_tdcs)
ssep_sham = extract_ssep(all_sham, specparam_sham)
ssep_baseline = extract_ssep(all_baseline, specparam_baseline)

In [ ]:
plot_ssep(ssep_baseline, ssep_sham, ssep_tacs, ssep_tdcs)

### Goal 1.4: Extract time-domain features from SSEP signals

**Analysis overview**: The `analyze_ssep()` function extracts three time-domain features from the filtered SSEP signals. Response latency measures the onset time of the SSEP using Hilbert envelope thresholding. Amplitude slope quantifies the rate of amplitude change by fitting a linear trend to the smoothed envelope. Autocorrelation at 1-cycle lag assesses oscillation consistency by correlating the signal with itself shifted by one period. The `plot_ssep_temporal()` function visualizes distributions of all three temporal features for each treatment condition compared to its paired baseline.

**Key parameters**:
- Response latency: 50% of maximum envelope threshold with 50-ms smoothing window
- Amplitude slope: Linear fit to Hilbert envelope over time
- Autocorrelation: Pearson correlation at lag = 1/peak_frequency

**Results**: The histograms show distributions of temporal features across all trials and subjects. Comparing post-treatment to baseline:
- **Response latency**: All conditions show rapid onset, reflecting the stimulus-locked nature of SSEPs, with tACS and tDCS potentially slowing the onset.
- **Amplitude slope**: All conditions show slightly negative slopes, indicating gradual amplitude decay over the stimulus window, consistent with adaptation.
- **Autocorrelation**: All conditions show high autocorrelation, reflecting sustained oscillatory consistency throughout the stimulus window.

In [ ]:
temporal_tacs = analyze_ssep(ssep_tacs)
temporal_tdcs = analyze_ssep(ssep_tdcs)
temporal_sham = analyze_ssep(ssep_sham)
temporal_baseline = analyze_ssep(ssep_baseline)

In [ ]:
plot_ssep_temporal(temporal_baseline, temporal_sham, temporal_tacs, temporal_tdcs)

### Goal 1.5: Extract cycle-by-cycle waveform features

**Analysis overview**: The `analyze_bycycle()` function computes cycle-by-cycle features from the raw signal at channel Cz using the bycycle algorithm. For each trial, individual oscillation cycles are detected and characterized by amplitude, period, rise/decay times, waveform symmetry, peak/trough sharpness, and burst rate. Burst detection uses relaxed thresholds optimized for stimulus-driven 40 Hz SSEPs. The `plot_bycycle()` function visualizes distributions of all eight bycycle features for each treatment condition compared to its paired baseline.

**Key parameters**:
- Frequency range: 30-50 Hz for cycle detection
- Waveform features: Computed from peak-to-trough dynamics within each cycle
- Burst thresholds: Amplitude fraction = 0.2, consistency = 0.6, monotonicity = 0.6, minimum cycles = 2

**Results**: The histograms show distributions of cycle-by-cycle features across all trials and subjects. Comparing post-treatment to baseline:
- **Amplitude**: tACS shows increased amplitude relative to baseline, consistent with enhanced gamma oscillations.
- **Period**: All conditions show periods centered around 25 ms, corresponding to the 40 Hz stimulus frequency.
- **Rise time**: All conditions show rise times centered near 12-13 ms (half period).
- **Decay time**: All conditions show decay times centered near 12-13 ms (half period).
- **Rise-decay symmetry**: All conditions show symmetry near 0.5, indicating balanced waveform shape.
- **Peak voltage**: Minimal differences across conditions.
- **Trough voltage**: tACS may lower the trough relative to baseline, suggesting sharpening.
- **Burst rate**: All conditions show very low burst rates. This likely reflects the steady-state nature of SSEPs, which are continuous stimulus-driven oscillations rather than intermittent bursts emerging from background activity.


In [ ]:
bycycle_tacs = analyze_bycycle(all_tacs, specparam_tacs)
bycycle_tdcs = analyze_bycycle(all_tdcs, specparam_tdcs)
bycycle_sham = analyze_bycycle(all_sham, specparam_sham)
bycycle_baseline = analyze_bycycle(all_baseline, specparam_baseline)

In [ ]:
plot_bycycle(bycycle_baseline, bycycle_sham, bycycle_tacs, bycycle_tdcs)

### Goal 1.6: Engineer features and reduce dimensionality

**Analysis overview**: Correlated features were identified and redundant variables removed to optimize the feature space for ML. The `rise_time` and `decay_time` features showed high correlation (r = 0.77) with `rise_decay_symmetry` and minimal discriminative value across conditions. These were dropped in favor of the symmetry metric. The `peak_voltage` feature was dropped due to minimal differences across conditions and redundancy with the `trough_voltage` feature (r = 0.78). To leverage the known effect of tACS on change from baseline and control for individual differences, baseline-relative features were computed for all stimulation trials. Each stimulation trial's features were then expressed as change from that subject's baseline, allowing single-trial classification while directly modeling the physiological effect of interest.

**Key parameters**:
- Dropped features: `rise_time`, `decay_time` (high correlation with `rise_decay_symmetry`, minimal between-condition variance); `peak_voltage` (redundant with `trough_voltage`, minimal condition differences)
- Baseline reference: Subject-specific median of each feature across baseline trials
- Feature transformation: Change from baseline (stim_trial - baseline_median) for all 14 features
- Final feature count: 14 baseline-relative features

**Results**: The baseline-relative feature engineering approach controls for between-subject variability while preserving single-trial resolution for classification. This transformation models the known physiological effect (tACS-induced change from baseline) and mirrors realistic deployment scenarios where a baseline calibration period precedes stimulation. The resulting features span spectral (`peak_frequency`, `peak_power`, `peak_bandwidth`, `peak_snr`, `aperiodic_offset`, `aperiodic_slope`), temporal (`response_latency`, `amplitude_slope`, `autocorr_1cycle`), and waveform morphology (`amplitude`, `period`, `rise_decay_symmetry`, `trough_voltage`, `burst_rate`) domains, expressed as changes that should differentiate tACS from tDCS and sham conditions. Dataset includes 6923 stimulation trials (tACS: 2332, tDCS: 2183, Sham: 2408) across 39 subjects, maintaining an appropriate features-to-samples ratio (14 features / 6923 trials = 0.002).

In [ ]:
# create df of all conditions and paired baselines
df = create_paired_df(
    specparam_baseline, specparam_sham, specparam_tacs, specparam_tdcs,
    temporal_baseline, temporal_sham, temporal_tacs, temporal_tdcs,
    bycycle_baseline, bycycle_sham, bycycle_tacs, bycycle_tdcs
)

print(f'Total: {len(df)} trials, {len(df["sid"].unique())} subjects')
print(f'  Baseline: {(df.condition == "Baseline").sum()} trials')
print(f'  tACS/tDCS/Sham: {len(df) - (df.condition == "Baseline").sum()} trials')
print(f'\nNumber of features: {len(df.columns[4:])}')
print(f'\n{df.head()}')

In [ ]:
# correlate all features to identify redundant variables
tmp = df.drop(columns = ['sid', 'tid', 'condition', 'group'])
tmp.corr()

In [ ]:
# drop redundant variables
df = df.drop(columns = ['rise_time', 'decay_time', 'peak_voltage'])
features = df.columns[4:]
print(f'Final number of features: {len(features)}')

In [ ]:
# compute baseline reference per subject
baseline_refs = df[df['condition'] == 'Baseline'].groupby('sid')[features].median()

# compute change from baseline for stimulation trials
stim_trials = df[df['condition'] != 'Baseline'].copy()
for sid in stim_trials['sid'].unique():
    mask = stim_trials['sid'] == sid
    stim_trials.loc[mask, features] = (
        stim_trials.loc[mask, features].values - baseline_refs.loc[sid].values
    )

stim_trials = stim_trials.drop(columns = 'condition')  # now redundant with group

print(f'Total: {len(stim_trials)} trials, {len(stim_trials["sid"].unique())} subjects')
print(f'  tACS: {(stim_trials.group == "tACS").sum()} trials')
print(f'  tDCS: {(stim_trials.group == "tDCS").sum()} trials')
print(f'  Sham: {(stim_trials.group == "Sham").sum()} trials')
print(f'\n{stim_trials.head()}')

### Goal 2.1: Compare models to classify stimulation condition

**Analysis overview**: Multiple ML models were compared to classify stimulation conditions (tACS, tDCS, Sham) using single-trial baseline-relative features with a 70/30 train/test split. This within-subject calibration strategy mirrors realistic deployment scenarios where baseline trials are collected before classification of stimulation trials. A train/test split approach was used rather than leave-one-subject-out cross-validation to account for individual differences in stimulation response. Four classification algorithms were evaluated: random forest, gradient boosting, logistic regression, and SVM with RBF kernel.

**Models tested**:
-  Random forest (200 trees, max_depth = 15, balanced class weights)
- Gradient boosting (200 estimators, max_depth = 6, learning_rate = 0.1)
- Logistic regression (L2 regularization, C = 1.0, balanced class weights, StandardScaler)
- SVM (RBF kernel, C = 1.0, balanced class weights, StandardScaler)

**Results**: Gradient boosting achieved the highest performance (balanced accuracy = 0.653), followed closely by random forest (0.626). Tree-based methods substantially outperformed linear models (logistic regression: 0.436, SVM: 0.530).

**Interpretation**: The ~2x improvement over chance (0.333) for 3-class classification demonstrates that within-subject baseline-relative features enable condition discrimination despite high trial-to-trial variability in single-trial EEG data. The superior performance of tree-based methods over linear models suggests complex, non-linear decision boundaries between conditions.

In [ ]:
results = compare_models(stim_trials, features)

### Goal 2.2: Optimize hyperparameters

**Analysis overview**: Gradient boosting hyperparameters were optimized using a 2-stage randomized search strategy. Stage 1 performed a coarse search across wide parameter ranges to identify promising regions of the hyperparameter space. Stage 2 performed a fine search with narrowed ranges centered on the best Stage 1 parameters. This approach balances computational efficiency with thorough exploration.

**Key parameters**:
- Stage 1 (coarse search): 30 iterations, wide ranges
  - n_estimators: 100-500
  - max_depth: 3-10
  - learning_rate: 0.01-0.30
  - min_samples_split: 10-50
  - min_samples_leaf: 5-30
  - subsample: 0.6-1.0
  - max_features: ['sqrt', 'log2', None]
- Stage 2 (fine search): 20 iterations, narrow ranges (+/- 50 n_estimators, +/- 1 max_depth, +/- 0.05 learning_rate, etc.)
- Cross-validation: 5-fold stratified CV on training set
- Scoring metric: Balanced accuracy

**Results**: The 2-stage search identified optimal hyperparameters: n_estimators = 384, max_depth = 9, learning_rate = 0.148, min_samples_split = 37, min_samples_leaf = 9, subsample = 0.974, max_features = None. The best CV score improved from 0.667 (Stage 1 coarse search) to 0.669 (Stage 2 fine search). Test set performance reached 0.658 balanced accuracy, representing a +0.004 improvement over default hyperparameters.

**Interpretation**: The modest gain suggests the default gradient boosting parameters were reasonably well-suited to this dataset, though optimization still provided measurable improvement. The optimized model showed balanced performance across all three classes (Sham: 0.70, tACS: 0.65, tDCS: 0.63 recall), indicating no systematic bias toward any condition.

In [ ]:
best_model, search_results = tune_gradient_boosting(stim_trials, features)

### Goal 2.3: Identify optimal features

**Analysis overview**: Feature importance was quantified using the trained gradient boosting model's built-in feature importance scores. All 14 features were ranked, and cumulative importance was calculated to identify thresholds. Performance was then evaluated across different feature subset sizes using the optimized hyperparameters to determine the optimal balance between model complexity and predictive performance.

**Key parameters**:
- Model: Gradient boosting with optimized hyperparameters
- Feature subsets tested: Top 5-14 features
- Importance metric: Gradient boosting feature importances (normalized to sum to 1.0)
- Evaluation: Balanced accuracy on held-out test set

**Results**: Feature importance analysis revealed `amplitude` (23.6%) as the dominant discriminative feature, followed by `burst_rate` (10.3%) and `autocorr_1cycle` (8.8%). Performance testing across feature subsets identified 13 features as optimal, achieving 0.668 balanced accuracy, a 0.01 improvement over using all 14 features. The optimal 13-feature model maintained balanced 3-class performance (Sham: 0.70, tACS: 0.66, tDCS: 0.64 recall).

**Interpretation**: The excluded feature (`peak_power`) contributed noise rather than signal, demonstrating that `amplitude` captures the gamma enhancement more directly at the single-trial level. The slight decline in performance from 13 to 14 features (0.668 to 0.656) demonstrated that dimensionality reduction improved generalization by eliminating noisy features.

In [ ]:
# rank features by importance
importances_df = rank_features(stim_trials, features, best_model)

In [ ]:
# test subsets of features
results_df = test_feature_subsets(
    stim_trials, features, importances_df,
    search_results['fine_best_params']
)

### Goal 2.4: Validate model performance

**Analysis overview**: Permutation testing was conducted to verify that the trained model learned real relationships between features and stimulation conditions rather than spurious patterns. In each permutation, training set condition labels were randomly shuffled, a new model was trained on the shuffled labels, and performance was evaluated on the held-out test set with real labels. This process was repeated 100 times to construct a null distribution of performance scores expected under the hypothesis that features contain no information about stimulation condition. The real model's performance was then compared against this null distribution to calculate a p-value.

**Key parameters**:
- Number of permutations: 100 (0.01 p-value precision)
- Model: Gradient boosting with optimized hyperparameters, 13 selected features
- Null hypothesis: Features contain no information about stimulation condition (expected performance near chance = 0.333)
- Statistical test: Proportion of permuted scores >= real score

**Results:** Real model balanced accuracy: 0.668, permuted model mean: 0.331 +/- 0.013, p-value < 0.01. The real model outperformed all permuted models, demonstrating significant learning of real physiological patterns. The permuted models performed at chance level (0.333), confirming that the baseline-relative features contain no discriminative information when the relationship between features and conditions is broken.

**Interpretation:** The 2.02x improvement of the real model over the null distribution demonstrates that the classifier learned true condition-specific signatures rather than spurious patterns and confirms that baseline-relative features contain discriminative information about stimulation conditions despite high single-trial variability.

In [ ]:
# set optimal 13 features
optimal_features = importances_df.head(13)['feature'].tolist()

# run permutation tests
perm_results = permutation_test(
    stim_trials,
    optimal_features,
    search_results['fine_best_params']
)